In [8]:
KUBECONFIG_PATH = "/home/junzhouh/distributed_debugger/SocialNetwork-serviceweaver/kubeconfig"
SERVICEWEAVER_POD_LABEL = "serviceweaver/app=gotosocial" # eg. "serviceweaver/app=server.out"

In [9]:

from kubernetes import client,config
config.load_kube_config(KUBECONFIG_PATH)
api_instance=client.CoreV1Api()

In [10]:
from uuid import uuid4
debug_container_name=f"ssh-debugger{uuid4()}"
debug_container_name

'ssh-debuggerd0d1eb4b-2f44-450b-a42d-fa863d51eb7e'

In [11]:
def setup_debug_container(pod_name: str, namespace: str = "default") -> None:

    debug_container = client.V1EphemeralContainer(
                    name=debug_container_name,
                    image="h21565897/distributeddebugger:144",
                    target_container_name="serviceweaver",
                    image_pull_policy="Always",
                    stdin=True,
                    tty=True,
                    security_context=client.V1SecurityContext(
                        privileged=True
                    )
                )
    patch_body = {
                "spec": {
                    "ephemeralContainers": [
                        debug_container
                    ]
                }
            }
            
            # Apply the patch
    api_instance.patch_namespaced_pod_ephemeralcontainers(
        name=pod_name,
        namespace="default",
        body=patch_body
    )

In [12]:
pods = api_instance.list_namespaced_pod(
            namespace="default", label_selector=SERVICEWEAVER_POD_LABEL)
len(pods.items)

3

In [13]:
for pod in pods.items:
    setup_debug_container(pod.metadata.name)
    print(f"Debug container injected for pod {pod.metadata.name}")

Debug container injected for pod weaver-main-5bb81ff6-e4a25d35-66497655df-scgh9
Debug container injected for pod weaver-mediarequesthandler-5bb81ff6-5be2b6d6-654948d669-pmxt8
Debug container injected for pod weaver-statusrequesthandler-5bb81ff6-0afde9d8-6d8bbf76b4-fvs7v
